# Regularization and Scaling and Polynomials, OH MY!
## Predicting MPG via car features

![](img/vroomvroom.jpg)

In [6]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score

import statsmodels.api as sm

import matplotlib.pyplot as plt

SyntaxError: invalid syntax (<ipython-input-6-92974c157c50>, line 1)

In [ ]:
df = pd.read_csv('auto-mpg.data')
df.drop(columns = 'junk', inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

## Features via Correlation

In [ ]:
df.corr()[['mpg']].sort_values(by='mpg', ascending = False)

In [ ]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

In [ ]:
model.summary()

# Creating Interaction Terms

In [3]:
poly = PolynomialFeatures()



In [ ]:
X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

# Apply Regularization to Aid in Feature Selection
## Re-define X, y from original dataframe to drop the constant from `statsmodels`

In [ ]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

## Re-create your polynomial features

In [ ]:
poly = PolynomialFeatures()


X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

## 🚂 Train, Test, Split 🤸🏻‍

## 👏🏻 Scale 👏🏻 Your 👏🏻 Data 👏🏻 

In [ ]:
ss = StandardScaler()

## RidgeCV

In [ ]:
ridge = RidgeCV()




In [ ]:
#training score


In [ ]:
#testing score


In [ ]:
#cross val score


In [ ]:
ridge.coef_

In [ ]:
pd.DataFrame(zip(X_poly_features, ridge.coef_), columns = ['feature', 'coefficient'] ).sort_values(by='coefficient')

### How do we interpret these coefficients?

### What features might you remove? How would you make that decision?

## Lasso

In [ ]:
lasso = LassoCV(max_iter = 10_000)



In [ ]:
#training score


In [ ]:
#testing score


In [ ]:
#cross val score


In [ ]:
#coefficients


In [ ]:
pd.DataFrame(zip(X_poly_features, lasso.coef_), columns = ['feature', 'coefficient'] ).sort_values(by='coefficient')

### How do we interpret these coefficients?

### What features might you remove? How would you make that decision?

# Extra, Streeeeeeetch
## Recreating the `statsmodels` `.summary()` output to get p-values for your features

### _Liberally lifted from and based on https://stackoverflow.com/a/42677750_

In [ ]:
from scipy import stats

In [ ]:
def model_summary(estimator, X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 737)
    
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model = estimator()
    if LassoCV:
        model = estimator(max_iter = 10_000)
    model.fit(X_train_sc,y_train)
    
    print(f'Training Score: {model.score(X_train_sc,y_train)}')
    print(f'Testing Score: {model.score(X_test_sc,y_test)}')
    print(f'Cross Validation Score: {cross_val_score(model, X_train_sc, y_train).mean()}\n')
    
    
    params = np.append(model.intercept_,model.coef_)
    predictions = model.predict(X_train_sc)

    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
#     newX = np.append(np.ones((len(X),1)), X, axis=1)
#     MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    print(myDF3)

In [ ]:
X = df.drop(['mpg', 'car_name'], axis = 1)
y = df['mpg']

poly = PolynomialFeatures()

X_poly = poly.fit_transform(X)
X_poly_features = poly.get_feature_names(X.columns)

X_poly = pd.DataFrame(X_poly, columns = X_poly_features)

In [ ]:
model_summary(LassoCV, X_poly, y)